In [1]:
!pip install requests beautifulsoup4 pandas openpyxl

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_tables(url):
    encoding = 'utf-8'
    # Send a GET request to the URL
    response = requests.get(url)
    response.encoding = encoding

    # Check if the request was successful
    if response.status_code == 200:
        html_content = response.text

        # Initialize lists to store scraped data
        school_data = []

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all tables with class="tablestyleA"
        tables = soup.find_all('table', class_='tablestyleA')

        return tables

In [46]:
from bs4 import BeautifulSoup
import pandas as pd

# Function to process each table element
def process_table(table):
    data = []
    rows = table.find_all('tr')

    print('===========================================TABLE START===========================================')

    # Skip the 1st and 2nd <tr> elements and take the 3rd <tr> element
    if len(rows) >= 3:
        second_row = rows[1]

        # Extracting district and school type information
        school_district = second_row.find('td', align='left').get_text().strip().split('\n')[0].strip()
        school_type = second_row.find('td', align='center').get_text().strip().split('\n')[0].strip()

        third_row = rows[2]
        cells = third_row.find_all('td', colspan="3")

        if cells:
            table = cells[0].find('table', attrs={'border': '1', 'bordercolor': '#000099'})
            if table:
                print('Table found')

                rows_inside = table.find_all('tr', recursive=False)

                ignore_first_row = True

                for row in rows_inside:
                    if ignore_first_row:
                        ignore_first_row = False
                        continue

                    cells = row.find_all('td', recursive=False)  # Only look for direct children

                    if len(cells) >= 5:

                        # Cell 1
                        school_soup = BeautifulSoup(str(cells[1]), 'html.parser')

                        # Mapping the extracted information into variables
                        # EN School Name
                        school_en_element = school_soup.find_all('tr')[0].find('td', class_='bodytxt').get_text()
                        school_name_en = school_en_element.split('\n')[0].strip()
                        print(f'school name en: {school_name_en}')
                        # EN Address
                        address_en_element = school_soup.find_all('tr')[1].find('td', class_='bodytxt').get_text()
                        address_en = address_en_element.split('\n')[1].strip()
                        print(f'address en: {address_en}')

                        # CN School Name
                        school_cn_element = school_soup.find_all('tr')[2].find('td', class_='bodytxt').get_text()
                        school_name_cn = school_cn_element.split('\n')[0].strip()
                        print(f'school name cn: {school_name_cn}')
                        # CN Address
                        address_cn_element = school_soup.find_all('tr')[3].find('td', class_='bodytxt').get_text()
                        address_cn = address_cn_element.split('\n')[0].strip()
                        print(f'address cn: {address_cn}')

                        # Cell 2
                        tel = cells[2].find('table').find('td', class_='bodytxt').text.strip().split(':')[1].strip()
                        print(f'tel: {tel}')
                        fax = cells[2].find('table').find_all('td', class_='bodytxt')[1].text.strip().split(':')[1].strip()
                        print(f'fax: {fax}')

                        # Cell 3
                        role_soup = BeautifulSoup(str(cells[3]), 'html.parser')
                        role1_element = role_soup.find_all('tr')[0].find('td', class_='bodytxt').get_text()
                        role2_element = role_soup.find_all('tr')[1].find('td', class_='bodytxt').get_text()

                        # Mapping the extracted information into variables
                        role1_title = role1_element.split('\n')[2].strip()
                        print(f'role1 title: {role1_title}')
                        role1_en = role1_element.split('\n')[4].strip()
                        print(f'role1 EN: {role1_en}')
                        role1_cn = role1_element.split('\n')[5].strip()
                        print(f'role1 CN: {role1_cn}')

                        role2_title = role2_element.split('\n')[0].strip()
                        print(f'role2 title: {role2_title}')
                        role2_en = role2_element.split('\n')[1].strip()
                        print(f'role2 EN: {role2_en}')
                        role2_cn = role2_element.split('\n')[3].strip()
                        print(f'role2 CN: {role2_cn}')

                        data.append({
                            'School Name EN': school_name_en,
                            'School Name CN': school_name_cn,
                            'District': school_district,
                            'School Type': school_type,
                            'Address EN': address_en,
                            'Address CN': address_cn,
                            'Tel': tel,
                            'Fax': fax,
                            'Contact 1 Title': role1_title,
                            'Contact 1 EN': role1_en,
                            'Contact 1 CN': role1_cn,
                            'Contact 2 Title': role2_title,
                            'Contact 2 EN': role2_en,
                            'Contact 2 CN': role2_cn,
                        })

                        print(data)

                        """
                        # Check for end of record
                        if cells and cells[0].text.strip() == '<!-- E04004: WebSite and Email -->':
                            break
                        """
    return data

def scrap(url):
    # List of tables containing page data
    list_of_tables = get_tables(url)
    #list_of_tables = get_tables("https://www.edb.gov.hk/en/student-parents/sch-info/sch-search/schlist-by-district/school-list-cw.html")
    #list_of_tables = [list_of_tables[0], list_of_tables[1]]

    # Process each table to extract data
    all_data = []
    for table_element in list_of_tables:
        if table_element is not None:
            print(type(table_element))
            soup = BeautifulSoup(str(table_element), 'html.parser')
            print(type(soup))
            table_data = process_table(soup)
            all_data.extend(table_data)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(all_data)

    return df

In [50]:
root_url = "https://www.edb.gov.hk/en/student-parents/sch-info/sch-search/schlist-by-district/"
result = pd.DataFrame()

# Send a GET request to the root URL
response = requests.get(root_url)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    # Find links to individual district pages
    district_links = [link.get("href") for link in soup.find_all("a", href=True)]

    prefix = "http://www.edb.gov.hk/en/student-parents/sch-info/sch-search/schlist-by-district/"

    for link in district_links:
        if prefix in link:
            data = scrap(link)
            result = pd.concat([result, data], ignore_index=True)

# Save the DataFrame to an Excel file
excel_file = 'school_info_extracted.xlsx'
result.to_excel(excel_file, index=False)

print(f"School information extracted and saved to {excel_file}.")

Streaming output truncated to the last 5000 lines.
[{'School Name EN': 'SHUNG TAK CATHOLIC ENGLISH COLLEGE', 'School Name CN': '天主教崇德英文書院', 'District': 'YUEN LONG             元朗區', 'School Type': 'AIDED SECONDARY SCHOOLS  資助中學', 'Address EN': '1 HUNG TAK ROAD\xa0HUNG SHUI KIU, YUEN LONG NEW TERRITORIES', 'Address CN': '新界元朗洪水橋洪德路１號', 'Tel': '24764263', 'Fax': '24734437', 'Contact 1 Title': 'Supervisor 校監:', 'Contact 1 EN': '-', 'Contact 1 CN': '', 'Contact 2 Title': 'Head of School 校長:', 'Contact 2 EN': 'DR TANG SUI SIM CECILIA', 'Contact 2 CN': '鄧瑞嬋博士'}, {'School Name EN': 'SKH BISHOP BAKER SECONDARY SCHOOL', 'School Name CN': '聖公會白約翰會督中學', 'District': 'YUEN LONG             元朗區', 'School Type': 'AIDED SECONDARY SCHOOLS  資助中學', 'Address EN': 'FUNG YAU STREET SOUTH\xa0YUEN LONG NEW TERRITORIES', 'Address CN': '新界元朗鳳攸南街', 'Tel': '24754778', 'Fax': '24799150', 'Contact 1 Title': 'Supervisor 校監:', 'Contact 1 EN': 'MR LAI DANIEL', 'Contact 1 CN': '賴錫璋先生', 'Contact 2 Title': 'Head of School